In [12]:
import pandas as pd

df = pd.read_csv('artists.csv')
df.head()

/var/folders/39/yqq6m0qn4rj4_j3d6xk0bv_h0000gn/T/ipykernel_6644/2926729350.py:3: DtypeWarning: Columns (2,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('artists.csv')


,mbid,artist_mb,artist_lastfm,country_mb,country_lastfm,tags_mb,tags_lastfm,listeners_lastfm,scrobbles_lastfm,ambiguous_artist
0,cc197bad-dc9c-440d-a5b5-d52ba2e14234,Coldplay,Coldplay,United Kingdom,United Kingdom,rock; pop; alternative rock; british; uk; brit...,rock; alternative; britpop; alternative rock; ...,5381567.0,360111850.0,False
1,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,Radiohead,United Kingdom,United Kingdom,rock; electronic; alternative rock; british; g...,alternative; alternative rock; rock; indie; el...,4732528.0,499548797.0,False
2,8bfac288-ccc5-448d-9573-c33ea2aa5c30,Red Hot Chili Peppers,Red Hot Chili Peppers,United States,United States,rock; alternative rock; 80s; 90s; rap; metal; ...,rock; alternative rock; alternative; Funk Rock...,4620835.0,293784041.0,False
3,73e5e69d-3554-40d8-8516-00cb38737a1c,Rihanna,Rihanna,United States,Barbados; United States,pop; dance; hip hop; reggae; contemporary r b;...,pop; rnb; female vocalists; dance; Hip-Hop; Ri...,4558193.0,199248986.0,False
4,b95ce3ff-3d05-4e87-9e01-c97b66af13d4,Eminem,Eminem,United States,United States,turkish; rap; american; hip-hop; hip hop; hiph...,rap; Hip-Hop; Eminem; hip hop; pop; american; ...,4517997.0,199507511.0,False


In [13]:
df2 = df[df['ambiguous_artist'] == False]
df2 = df2[['tags_mb', 'tags_lastfm', 'artist_mb']]
df2['tags_mb'] = df2['tags_mb'].str.lower()
df2 = df2[df2['tags_mb'].apply(lambda x: isinstance(x, str))]
df2['tags_mb'] = df2['tags_mb'].str.replace(' ', '')
df2['tags_mb'] = df2['tags_mb'].str.replace(';', ' ')
df2 = df2.dropna(subset=['tags_mb'])

print(df2.shape)
df2.head()

(115472, 3)


,tags_mb,tags_lastfm,artist_mb
0,rock pop alternativerock british uk britanniqu...,rock; alternative; britpop; alternative rock; ...,Coldplay
1,rock electronic alternativerock british grunge...,alternative; alternative rock; rock; indie; el...,Radiohead
2,rock alternativerock 80s 90s rap metal america...,rock; alternative rock; alternative; Funk Rock...,Red Hot Chili Peppers
3,pop dance hiphop reggae contemporaryrb electro...,pop; rnb; female vocalists; dance; Hip-Hop; Ri...,Rihanna
4,turkish rap american hip-hop hiphop hiphop mid...,rap; Hip-Hop; Eminem; hip hop; pop; american; ...,Eminem


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(df2['tags_mb'])

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vectorized)

: 

In [ ]:
df3 = pd.DataFrame(similarities, columns=df2['artist_mb'], index=df2['artist_mb']).reset_index()

df3.head()

artist_mb,artist_mb,Coldplay,Radiohead,Red Hot Chili Peppers,Rihanna,Eminem,The Killers,Kanye West,Nirvana,Muse,...,Jeff Tweedy,9th Wonder,Mnemic,Natalie La Rose,Tommy Tutone,Free Energy,Battlelore,Swallow the Sun,Sander Kleinenberg,Spın̈al Tap
0,Coldplay,1.000000,0.321509,0.059263,0.044697,0.000000,0.111934,0.032851,0.050093,0.199413,...,0.114890,0.000000,0.0,0.000000,0.205153,0.205153,0.000000,0.000000,0.0,0.0
1,Radiohead,0.321509,1.000000,0.025787,0.000000,0.000000,0.043630,0.000000,0.069318,0.153751,...,0.073206,0.000000,0.0,0.000000,0.087565,0.087565,0.000000,0.000000,0.0,0.0
2,Red Hot Chili Peppers,0.059263,0.025787,1.000000,0.024938,0.068340,0.162413,0.019336,0.137383,0.187110,...,0.127501,0.000000,0.0,0.000000,0.100046,0.100046,0.054839,0.052911,0.0,0.0
3,Rihanna,0.044697,0.000000,0.024938,1.000000,0.045795,0.000000,0.084322,0.000000,0.000000,...,0.000000,0.119072,0.0,0.154153,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,Eminem,0.000000,0.000000,0.068340,0.045795,1.000000,0.017397,0.403735,0.010825,0.000000,...,0.034487,0.219922,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0


In [ ]:
input_artist = "Coldplay"
recommendations = pd.DataFrame(df3.nlargest(11,input_artist)['artist_mb'])
recommendations = recommendations[recommendations['artist_mb']!=input_artist]
print(recommendations)

            artist_mb
4157   Noel Gallagher
1674        Ian Brown
3530    Jarvis Cocker
98              Keane
2833    The Bluetones
4117        Catatonia
4957    The Seahorses
123     Kaiser Chiefs
912        Starsailor
738   Vanessa Carlton
